## Combine and Clean Raw Files

> **Project:** Houston 311 Detector  
> **Author:** Mojoolu (Mojo) Roberts  
> **Environment:** `.venv`  
> **Data Contract:** All notebooks read from raw Houston 311 data that can be downloaded here (link)

In [3]:
import pandas as pd 
import os
from pathlib import Path

## Combine Pre-2021 Raw 311 Files

**Purpose**  
Load and combine all pre-2021 Houston 311 raw data files into a single dataset.

**Input**
- Folder: `data/raw/pre2021/`
- Files: pipe-delimited (`|`) raw text/CSV exports

**Processing Details**
- Reads all files in the folder
- Uses UTF-8 with character replacement to handle encoding issues
- Skips malformed lines to prevent ingestion failures
- Appends each file into a unified list of DataFrames

**Output**
- `data/middle/pre2021.csv`  
- One row per original service request (no aggregation)

**Notes**
- This step preserves raw structure for downstream cleaning
- Column normalization and validation happen later


In [7]:
data_folder = "/Users/mojo/Documents/MyProjects/311Detector/data"
input_folder = f"{data_folder}/raw/pre2021"
output_csv = Path(f"{data_folder}/middle/pre2021.csv")
output_csv.parent.mkdir(parents=True, exist_ok=True)

pre_2021_dfs = []

for filename in os.listdir(input_folder):
    file_path = os.path.join(input_folder, filename)
    df = pd.read_csv(file_path, sep='|', engine='python', encoding="utf-8", encoding_errors="replace", on_bad_lines='skip')
    pre_2021_dfs.append(df)

pre_2021 = pd.concat(pre_2021_dfs, ignore_index=True)

pre_2021.to_csv(output_csv, index=False)

pre_2021.head()

CASE NUMBER                                       SR LOCATION  \
0  12091834-101002444724                    7237 RHOBELL, HOUSTON TX 77016   
1  12091835-101002444725                   7613 APPLETON, HOUSTON TX 77022   
2           101002444726  Intersection 3900 S GESSNER RD&10000 WESTPARK DR   
3           101002444727                     1411 BONNER, HOUSTON TX 77007   
4    169116-101002444728                                               NaN   

          COUNTY DISTRICT                              NEIGHBORHOOD  \
0         HARRIS        B              EAST LITTLE YORK / HOMESTEAD   
1         Harris        H                       NORTHSIDE/NORTHLINE   
2  Harris County        F                                  MID WEST   
3         HARRIS        C  WASHINGTON AVENUE COALITION / MEMORIAL P   
4        Unknown  Unknown                                   Unknown   

          TAX ID TRASH QUAD RECYCLE QUAD TRASH DAY HEAVY TRASH DAY  ...  \
0  0825260000007         NE           NW   TUESDAY     2nd Tuesday  ...   
1  0710210010005         NE           NW    MONDAY      3rd Monday  ...   
2            NaN        NaN          NaN       NaN             NaN  ...   
3  0221020000009         NW           NW    MONDAY   1st Wednesday  ...   
4        Unknown    Unknown      Unknown   Unknown         Unknown  ...   

              DUE DATE          DATE CLOSED OVERDUE  \
0  2017-01-13 00:01:48  2017-01-01 11:20:02  -11.53   
1  2017-01-13 00:07:29  2017-01-01 04:50:02  -11.80   
2  2017-01-02 00:14:04  2017-01-01 00:23:57   -0.99   
3  2017-01-08 00:17:23  2017-01-04 14:01:37   -3.43   
4  2017-01-03 08:00:00  2017-01-01 14:30:33   -1.73   

                                               Title             x  \
0                          Fire Hydrant-101002444724  3.142094e+06   
1                          Fire Hydrant-101002444725  3.119896e+06   
2            Traffic Signal Maintenance-101002444726  3.066828e+06   
3  Code Violation Report for Multi-Family Housing...           NaN   
4                                     Other - 169116  3.103546e+06   

              y     LATITUDE     LONGITUDE Channel Type  \
0  1.387764e+07  29.85674295  -95.29694519     Voice In   
1  1.386738e+07  29.83044921  -95.36794687     Voice In   
2  1.382635e+07  29.72209924  -95.53907768     Voice In   
3           NaN  29.77385877  -95.40257152          WEB   
4  1.385522e+07      Unknown       Unknown          WAP   

     Bud1            %                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

## Combine Post-2021 Raw 311 Files

**Purpose**  
Load and combine all post-2021 Houston 311 raw data files into a single dataset.

**Input**
- Folder: `data/raw/post2021/`
- Files: pipe-delimited (`|`) raw text/CSV exports

**Processing Details**
- Reads all files in the folder
- Uses latin-1 with character replacement to handle encoding issues
- Skips malformed lines to prevent ingestion failures
- Appends each file into a unified list of DataFrames

**Output**
- `data/middle/post2021.csv`  
- One row per original service request (no aggregation)

**Notes**
- This step preserves raw structure for downstream cleaning
- Column normalization and validation happen later


In [9]:
input_folder = f"{data_folder}/raw/post2021"
output_csv = Path(f"{data_folder}/middle/post2021.csv")
output_csv.parent.mkdir(parents=True, exist_ok=True)

post_2021_dfs = []

for filename in os.listdir(input_folder): 
    file_path = os.path.join(input_folder, filename) 
    df = pd.read_csv(file_path, sep='|', engine='python', encoding='latin-1', on_bad_lines='skip') 
    post_2021_dfs.append(df)
     
post_2021 = pd.concat(post_2021_dfs, ignore_index=True)
post_2021 = post_2021.drop(post_2021.columns[0], axis=1)

# Convert date columns safely
date_cols = ["Created Date Local", "Closed Date"]
for col in date_cols:
    post_2021[col] = pd.to_datetime(
        post_2021[col],
        errors='coerce',  # invalid dates become NaT
        format=None,
    )

# Inspect rows with invalid dates
for col in date_cols:
    invalid = post_2021[post_2021[col].isna()]
    print(f"{col} - rows with invalid dates: {len(invalid)}")


post_2021.to_csv(output_csv, index=False)
post_2021.head() 

Created Date Local - rows with invalid dates: 5
Closed Date - rows with invalid dates: 245453


,365 Case Number,Case Number,Incident Address,Latitude,Longitude,Status,Created Date Local,Closed Date,Title,Incident Case Type,...,Heavy Trash Quadrant,Queue,ETJ,SLA Name,Channel,Extract Date,Latest Case Notes,Sample Case Confilcts Notes,Description,Resolution Notes
0,2300751091,2300751091,15806 BOONRIDGE RD Houston Texas 77053,29.5936,-95.4560,Service Completed,2023-01-31 17:59:49,2023-02-14 08:27:00,Container Repair - 2300751091 - 15806 BOONRIDG...,Container Repair,...,SW,SWM Collections,FULL,10 Calendar Days Deactivated,Phone,2026-01-13 17:56:51.5730000,A service request has been completed at 15806 ...,NaN,bottom is broken,A service request has been completed at 15806 ...
1,2300751090,2300751090,6526 LINDEN ST Houston Texas 77087,29.7079,-95.3014,SWM Can Delivered,2023-01-31 17:58:55,2023-02-13 15:21:00,Add a Can - 2300751090 - 6526 LINDEN ST,Add a Can,...,SE,SWM GenSupportServ,FULL,14 Calendar Days Deactivated,Phone,2026-01-13 17:56:51.5730000,As per A Knighten #1374457 2-13-23,NaN,NaN,As per A Knighten #1374457 2-13-23
2,2300751089,20296133-2300751089,7820 E MAGNOLIA ST Houston Texas 77012,29.7190,-95.2812,Service Completed,2023-01-31 17:58:34,2023-02-02 13:41:00,Sewer Wastewater - 20296133-2300751089 - 7820 ...,Sewer Wastewater,...,SE,NaN,FULL,1 Calendar Day Deactivated,Phone,2026-01-13 17:56:51.5730000,2/2/2023 1:40:28 PM - Case # 20296133 -2300751...,NaN,NaN,2/2/2023 1:40:28 PM - Case # 20296133 -2300751...
3,2300751088,2300751088,9355 COLLEEN RD Houston Texas 77080,29.8251,-95.5247,Service Completed,2023-01-31 17:55:37,2023-02-03 17:11:00,Container Repair - 2300751088 - 9355 COLLEEN RD,Container Repair,...,NE,SWM Collections,FULL,10 Calendar Days Deactivated,Phone,2026-01-13 17:56:51.5730000,A service request has been completed at 9355 C...,NaN,trash truck damged the wheels. wheels need rep...,A service request has been completed at 9355 C...
4,2300751086,2300751086,5722 BEALL ST Houston Texas 77091,29.8498,-95.4180,Service Completed,2023-01-31 17:51:19,2023-02-01 12:54:00,Non Residential Collection Service NEW - 23007...,Non Residential Collection Service NEW,...,NE,SWM GenSupportServ,FULL,66 Calendar Days Deactivated,Phone,2026-01-13 17:56:51.5730000,Residential - This does not qualify as non res...,NaN,NaN,Residential - This does not qualify as non res...


## Select and Align Canonical Columns

**Purpose**  
Reduce pre-2021 and post-2021 datasets to a common set of fields so they can be merged safely.

**Context**
- Column names differ before and after the City of Houston schema change
- This step enforces a shared logical schema without renaming yet

**Pre-2021 Columns Kept**
- Case identifier
- Neighborhood / department hierarchy
- Service request type
- Created and closed dates
- Latitude / longitude

**Post-2021 Columns Kept**
- Same logical fields under updated column names

**Notes**
- No data cleaning or normalization happens here
- Renaming and type coercion are handled in later steps


In [10]:
pre_keep = ['CASE NUMBER', 'NEIGHBORHOOD', 'DEPARTMENT','DIVISION', 'SR TYPE', 'SR CREATE DATE', 
            'DATE CLOSED','LATITUDE', 'LONGITUDE']
post_keep = ['Case Number', 'Customer SuperNeighborhood', 'Department', 'Division', 
             'Incident Case Type', 'Created Date Local', 'Closed Date','Latitude', 'Longitude']

pre_df = pre_2021[pre_keep]
post_df = post_2021[post_keep]

## Normalize Column Names to Canonical Schema

**Purpose**  
Standardize column names across pre-2021 and post-2021 datasets to enable safe concatenation.

**Canonical Schema**
- `CASE NUMBER`
- `NEIGHBORHOOD`
- `DEPARTMENT`
- `DIVISION`
- `CASE TYPE`
- `CREATED DATE`
- `CLOSED DATE`
- `LATITUDE`
- `LONGITUDE`

**Details**
- Pre-2021 columns are renamed where needed
- Post-2021 columns are fully mapped to the canonical schema
- No values are modified in this step

**Result**
- Both DataFrames now share identical column names and semantics


In [11]:
pre_df = pre_df.rename(columns={
    'SR TYPE': 'CASE TYPE',
    'SR CREATE DATE': 'CREATED DATE',
    'DATE CLOSED': 'CLOSED DATE',
})

post_df = post_df.rename(columns={
    'Case Number': 'CASE NUMBER',
    'Customer SuperNeighborhood': 'NEIGHBORHOOD',
    'Department': 'DEPARTMENT',
    'Division': 'DIVISION',
    'Incident Case Type': 'CASE TYPE',
    'Created Date Local': 'CREATED DATE',
    'Closed Date': 'CLOSED DATE',
    'Latitude': 'LATITUDE',
    'Longitude': 'LONGITUDE',
})

## Merge Datasets and Persist Combined Output

**Purpose**  
Combine normalized pre-2021 and post-2021 311 records into a single unified dataset.

**Processing**
- Concatenate both DataFrames using the canonical schema
- Reset index to ensure continuity across eras
- Perform a quick structural check using `DataFrame.info()`

**Output**
- `data/middle/Combined_Houston_311.csv`

**Guarantees**
- One row per service request
- Consistent column names and meanings across all records
- No aggregation or filtering applied

**Next Step**
- Downstream notebooks consume this file for cleaning, feature engineering, and modeling

In [14]:
combined_df = pd.concat([pre_df, post_df], ignore_index=True)
output_file = Path(f"{data_folder}/middle/Combined_Houston_311.csv")
combined_df.to_csv(output_file, index=False)

print(combined_df.info())
combined_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3888750 entries, 0 to 3888749
Data columns (total 9 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   CASE NUMBER   object
 1   NEIGHBORHOOD  object
 2   DEPARTMENT    object
 3   DIVISION      object
 4   CASE TYPE     object
 5   CREATED DATE  object
 6   CLOSED DATE   object
 7   LATITUDE      object
 8   LONGITUDE     object
dtypes: object(9)
memory usage: 267.0+ MB
None


,CASE NUMBER,NEIGHBORHOOD,DEPARTMENT,DIVISION,CASE TYPE,CREATED DATE,CLOSED DATE,LATITUDE,LONGITUDE
0,12091834-101002444724,EAST LITTLE YORK / HOMESTEAD,PWE Public Works Engineering,PU Public Utilities,Fire Hydrant,2017-01-01 00:01:48,2017-01-01 11:20:02,29.85674295,-95.29694519
1,12091835-101002444725,NORTHSIDE/NORTHLINE,PWE Public Works Engineering,PU Public Utilities,Fire Hydrant,2017-01-01 00:07:29,2017-01-01 04:50:02,29.83044921,-95.36794687
2,101002444726,MID WEST,PWE Public Works Engineering,Traffic Operations,Traffic Signal Maintenance,2017-01-01 00:14:04,2017-01-01 00:23:57,29.72209924,-95.53907768
3,101002444727,WASHINGTON AVENUE COALITION / MEMORIAL P,PWE Public Works Engineering,PDS Planning Development Services,MultiFamily Habitability Violation,2017-01-01 00:17:23,2017-01-04 14:01:37,29.77385877,-95.40257152
4,169116-101002444728,Unknown,311 HelpLine,311 Call Handling,Unclassified 311 Web Request,2017-01-01 00:18:07,2017-01-01 14:30:33,Unknown,Unknown
